In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/sesac_final_project

/content/drive/MyDrive/sesac_final_project


In [1]:
# 기존 설정
!pip3 install -q -U transformers==4.38.2
!pip3 install -q -U datasets==2.18.0
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.9.0
!pip3 install -q -U trl==0.7.11
!pip3 install -q -U accelerate==0.27.2

import re
from typing import Any, Optional

import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    pipeline,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from huggingface_hub import notebook_login

In [4]:
ls

04_26_gemma_plus_mlflow.ipynb  job_ICT.csv    outputs/             personality_SM.csv
databricks-uri/                job_MM.csv     personality_ARD.csv  requirements.txt
dataset.zip                    job_PS.csv     personality_BM.csv   requirement.txt
gemma-2b-it-sum-ko/            job_RND.csv    personality_ICT.csv  use_data.csv
gemma_to_train.ipynb           job_SM.csv     personality_MM.csv   use_langchaing_model.ipynb
job_ARD.csv                    lora_adapter/  personality_PS.csv   가져가셈.csv
job_BM.csv                     mlruns/        personality_RND.csv


In [5]:
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [6]:

# 양자화 설정을 불러오는 대신 PEFT 설정을 적용
lora_config = LoraConfig(
    r=6,
    lora_alpha=8,
    lora_dropout=0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)



bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [10]:
from huggingface_hub import notebook_login
notebook_login()

In [11]:
# 오늘은 7b다
#google/gemma-1.1-7b-it
#google/gemma-7b-it
# 위 두가지 버전이 있지만 이를 1.1버전이 더 향상된 버전이라 하기에 1.1 7b로 진행


In [12]:
BASE_MODEL = 'google/gemma-1.1-7b-it'

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)
tokenizer.padding_side = 'right'

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

# 데이터 전처리단

In [13]:
raw_data = pd.read_csv('가져가셈.csv', encoding='utf-8')

In [14]:
import re

def remove_extra_spaces_and_truncate(text):
    # '.'을 기준으로 앞 뒤에 공백이 3개 이상인 경우 해당 '.'을 기준으로 뒤의 문장을 제거하고 앞의 문장을 반환
    result = re.sub(r'\s{3,}.*', '', text)

    # '.'이 반복되면 가장 앞에 있는 한개의 '.'만 남기도록 처리
    result = re.sub(r'\.+', '.', result)


    return result
## remove_extra_spaces_and_truncate 함수를 적용하여 새로운 열을 생성
# kobart_data['Kobart-summary'] = kobart_data['Kobart-summary'].apply(remove_extra_spaces_and_truncate)
for index in range(len(raw_data)):
  raw_data['Kobart-summary'][index] = remove_extra_spaces_and_truncate(raw_data['Kobart-summary'][index])

In [15]:
train_data = raw_data.copy()
train_dataset = Dataset.from_pandas(train_data)


In [16]:
train_dataset

Dataset({
    features: ['Question', 'Kobart-summary'],
    num_rows: 100
})

# 학습단 - 이부분이 mlflow 적용하는 부분이 될것임.


In [17]:
!pip install mlflow

In [18]:
import mlflow

In [19]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)


def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['Kobart-summary'])):
        prompt_list.append(r"""<bos><start_of_turn>user
첫번째 문장은 사용자에 대한 정보입니다. 다음 글을 기반으로 꼬리 질문을 생성해주세요 단 질문은 해당 글에서 정답을 찾을수 있으면 안됩니다:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example['Kobart-summary'][i], example['Question'][i]))
    return prompt_list

In [20]:
mlflow.set_experiment("MLflow_gemma")
'''
현재 set : <Experiment: artifact_location='file:///content/drive/MyDrive/sesac_final_project/mlruns/970434065738294307', creation_time=1714097510456, experiment_id='970434065738294307', last_update_time=1714097510456, lifecycle_stage='active', name='MLflow gemma', tags={}>
이건 이 코드 돌릴 때 마다 바뀜.
'''


"\n현재 set : <Experiment: artifact_location='file:///content/drive/MyDrive/sesac_final_project/mlruns/970434065738294307', creation_time=1714097510456, experiment_id='970434065738294307', last_update_time=1714097510456, lifecycle_stage='active', name='MLflow gemma', tags={}>\n이건 이 코드 돌릴 때 마다 바뀜.\n"

In [21]:
from datetime import datetime

In [22]:
datetime.now()

datetime.datetime(2024, 4, 29, 1, 38, 1, 344950)

## 한번에 가능한가 트래킹 기능 추가

In [ ]:
# 외부서버에서 돌려주세요 ulr 주소입니다. !mlflow server --host 0.0.0.0 --port 5000
# ngrok http 5000


# 로컬 주소 link = "172.16.217.47:5000"

[2024-04-26 05:43:06 +0000] [8224] [INFO] Starting gunicorn 21.2.0
[2024-04-26 05:43:06 +0000] [8224] [INFO] Listening at: http://0.0.0.0:5000 (8224)
[2024-04-26 05:43:06 +0000] [8224] [INFO] Using worker: sync
[2024-04-26 05:43:06 +0000] [8225] [INFO] Booting worker with pid: 8225
[2024-04-26 05:43:07 +0000] [8226] [INFO] Booting worker with pid: 8226
[2024-04-26 05:43:07 +0000] [8227] [INFO] Booting worker with pid: 8227
[2024-04-26 05:43:07 +0000] [8228] [INFO] Booting worker with pid: 8228

Aborted!
[2024-04-26 05:43:25 +0000] [8224] [INFO] Handling signal: int
[2024-04-26 05:43:25 +0000] [8228] [INFO] Worker exiting (pid: 8228)
[2024-04-26 05:43:25 +0000] [8225] [INFO] Worker exiting (pid: 8225)
[2024-04-26 05:43:25 +0000] [8227] [INFO] Worker exiting (pid: 8227)
[2024-04-26 05:43:25 +0000] [8226] [INFO] Worker exiting (pid: 8226)
[2024-04-26 05:43:25 +0000] [8224] [INFO] Shutting down: Master


In [24]:
train_dataset

Dataset({
    features: ['Question', 'Kobart-summary'],
    num_rows: 100
})

In [26]:
import mlflow
from datetime import datetime
from transformers import TrainingArguments, Trainer

# MLflow 트래킹 서버의 URI를 설정합니다. Ngrok이 제공하는 URL을 사용합니다.
mlflow.set_tracking_uri('https://a8dd-180-68-46-170.ngrok-free.app')

# MLflow 실험의 이름을 설정합니다.
mlflow.set_experiment("Default")

# 트레이닝을 위한 인자들을 설정합니다.
training_args = TrainingArguments(
    output_dir="outputs/mlflow",
    max_steps=100,  # 최대 반복 횟수
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    warmup_steps=0.03,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=100,
    push_to_hub=False,
)


In [ ]:
import time
# MLflow 로깅을 시작합니다.
with mlflow.start_run():

    time.sleep(4) # 2초정도 instance 만드는 동안 기다려 줘야합니다 (여유를 가져야 train 담길 id가 제대로 생성됨.)
    # 현재 실험에 대한 파라미터를 로그합니다.
    mlflow.log_params({
        "learning_rate": training_args.learning_rate,
        "batch_size": training_args.per_device_train_batch_size,
        "max_steps": training_args.max_steps
    })

    # 트레이너를 설정하고 모델, 데이터셋, 그리고 기타 설정을 초기화합니다.
    # 여기서 model, train_dataset는 미리 정의되어야 하며 해당 설정에 맞게 수정해야 합니다.
    trainer = SFTTrainer(
        model=model,
        train_dataset=train_dataset,
        max_seq_length=512,
        args=training_args,
        peft_config=lora_config,
        formatting_func=generate_prompt,
    )

    # 모델 트레이닝을 시작합니다.
    trainer.train()

    # 트레이닝이 완료된 후 모델을 MLflow에 로그합니다.
    mlflow.pytorch.log_model(trainer.model, "model")

mlflow.end_run()


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Step,Training Loss
100,1.904500
200,0.261200
300,0.100100
400,0.060100
500,0.048400


Checkpoint destination directory outputs/mlflow/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


PicklingError: Cannot pickle a prepared model with automatic mixed precision, please unwrap the model with `Accelerator.unwrap_model(model)` before pickling it.

# mlflow에 업로드 된거 확인하는 단?

In [ ]:
model.config

GemmaConfig {
  "_name_or_path": "google/gemma-1.1-7b-it",
  "architectures": [
    "GemmaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 2,
  "eos_token_id": 1,
  "head_dim": 256,
  "hidden_act": "gelu",
  "hidden_activation": "gelu_pytorch_tanh",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 24576,
  "max_position_embeddings": 8192,
  "model_type": "gemma",
  "num_attention_heads": 16,
  "num_hidden_layers": 28,
  "num_key_value_heads": 16,
  "pad_token_id": 0,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "float16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 

In [ ]:
prompt_template = "안녕하세요, 저는 면접관입니다. 다음 질문에 대한 답변을 부탁드립니다. Q: {prompt}"

In [ ]:
# MLflow 실행 시작 및 모델 로깅
import mlflow

# 이전에 시작된 MLflow 실행의 ID를 가져옴
last_run_id = mlflow.last_active_run().info.run_id
# 패딩이 제거된 토크나이저 생성
tokenizer_no_pad = AutoTokenizer.from_pretrained(BASE_MODEL, add_bos_token=True)

with mlflow.start_run(run_id=last_run_id):
    # 모델 구성 매개변수를 MLflow에 기록
    mlflow.log_params(model.config.to_dict())
    # 모델과 토크나이저를 MLflow에 로깅
    mlflow.transformers.log_model(
        transformers_model={"model": trainer.model, "tokenizer": tokenizer_no_pad},
        prompt_template=prompt_template,
        artifact_path="model",
    )


2024/04/26 03:52:44 INFO mlflow.transformers: Overriding save_pretrained to False for PEFT models, following the Transformers behavior. The PEFT adaptor and config will be saved, but the base model weights will not and reference to the HuggingFace Hub repository will be logged instead.
2024/04/26 03:52:44 INFO mlflow.transformers: Skipping saving pretrained model weights to disk as the save_pretrained is set to False. The reference to HuggingFace Hub repository google/gemma-1.1-7b-it will be logged instead.
2024/04/26 03:52:44 INFO mlflow.transformers: text-generation pipelines saved with prompt templates have the `return_full_text` pipeline kwarg set to False by default. To override this behavior, provide a `model_config` dict with `return_full_text` set to `True` when saving the model.
2024/04/26 03:52:44 WARNING mlflow.utils.requirements_utils: Found torch version (2.2.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.2.1

In [ ]:
last_run_id

'0bf1fbe2fc89419bb4c2d4b53cc8ec60'

#  로드해보기/

In [ ]:
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 9.2 MB/s eta 0:00:00


In [ ]:
import mlflow

In [ ]:
# MLflow에 저장된 모델을 로드
mlflow_model = mlflow.pyfunc.load_model("runs:/0bf1fbe2fc89419bb4c2d4b53cc8ec60/model")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# 모델 추론 및 결과 출력
# 테스트 프롬프트 정의
inputs = 'RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 개인주의적인 사람이지만 협업을 할 때는 사교성이 좋은 편이라고 생각합니다. 유대감을 키워서 협업을 해야만 훨씬 더 어 제대로 된 결과물이 나왔기 때문입니다.'

test_prompt = f"""

첫번째 문장은 사용자에 대한 정보입니다. 다음 text을 기반으로 꼬리 질문을 생성해주세요 단 질문은 해당 글에서 정답을 찾을수 있으면 안됩니다:


text:
{inputs}

질문:

"""
# # 저장된 MLflow 모델을 사용하여 테스트 프롬프트에 대한 추론 수행
# generated_query = mlflow_model.predict(test_prompt)[0]
# display_table({"prompt": test_prompt, "generated_query": generated_query})

In [ ]:
# Encode the input prompt
inputs = tokenizer.encode(test_prompt, return_tensors='pt')

# Adjust the max_new_tokens parameter to control the generation length
generated_tokens = model.generate(inputs, max_new_tokens=200)  # Adjust this value as needed

# Decode the generated tokens to a string
generated_query = tokenizer.decode(generated_tokens[0])

# Output the result
print(generated_query)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<bos>

첫번째 문장은 사용자에 대한 정보입니다. 다음 text을 기반으로 꼬리 질문을 생성해주세요 단 질문은 해당 글에서 정답을 찾을수 있으면 안됩니다:


text:
RND 직무의 인성면접에 참가하게 된 신입 지원자입니다. 저는 개인주의적인 사람이지만 협업을 할 때는 사교성이 좋은 편이라고 생각합니다. 유대감을 키워서 협업을 해야만 훨씬 더 어 제대로 된 결과물이 나왔기 때문입니다.

질문:

RND 직무의 신입 지원자가 개인주의적인 사람이지만 협업을 할 때는 사교성이 좋은 편이라고 생각한다는 주장을 지원하는 어떤 이유가 있나요?


**답: 유대감을 키워서 협업을 해야만 훨씬 더 어 제대로 된 결과물이 나왔기 때문입니다.**<eos>


In [ ]:
inputs = 'SM 직무의 인성면접에 참가하게 된 신입 지원자입니다. 음 제가 팀을 이루어서 스터디를 하고 있는 것은 없지만 제 개인적으로는 아이티 분야와 인공 지능 분야에 대해서 어 어떤 공부를 지금 시작해서 해 나가고 있습니다. 제가 바뀌어 나가는 사회 세상에 대해서 관심을 가지게 되다 보니까 아이티 분야 인공 지능 분야가 굉장히 앞으로는 필요한 분야고 제가 따로 공부를 해두지 않으면 따라가기 어려운 분야라는 생각이 들어서 공부를 해야겠다고 결심을 하게 됐고 하나씩 하나씩 공부를 해 나가고 있습니다. 바뀌어나가고 알 수 없는 단어들이나 알 수 없는 변화들이 계속 제 주위에 나타나는데 그것을 외면하면서 살 수는 없더라구요. 그래서 하나씩 하나씩 제가 처음 들어오는 단어에 대해서 관심을 가지고 그것이 무엇인지 찾아가다 보니까 결국은 깊이 있는 공부가 필요하다는 생각이 들었고 지금은 깊이 있게 공부를 하기 위해서 우선 관련 분야 책을 읽는 것으로 공부를 시작하고 있습니다.,저는 아이티 분야와 인공 지능 분야에 대해 따로 공부를 해두지 않으면 따라가기 어려운 분야라는 생각이 들어서 공부를 해야겠다고 결심을 하게 됐습니다. 알 수 없는 변화들이 계속 나타나는데 외면하면서 살 수는 없어서 깊이 있게 공부를 하기 위해서 우선 관련 분야 책을 읽는 것으로 공부를 시작하고 있습니다.'

In [ ]:

test_prompt = f"""

첫번째 문장은 사용자에 대한 정보입니다. 다음 text을 기반으로 꼬리 질문을 생성해주세요 단 질문은 해당 글에서 정답을 찾을수 있으면 안됩니다:
조건이 있습니다.
text에서 정답을 찾으면 안됩니다.

text:
{inputs}

질문:

"""

In [ ]:
import torch
del generated_query
del generated_tokens
del inputs
torch.cuda.empty_cache()  # GPU 캐시 메모리 클리어


In [ ]:
# Encode the input prompt
inputs = tokenizer.encode(test_prompt, return_tensors='pt')

# Adjust the max_new_tokens parameter to control the generation length
generated_tokens = model.generate(inputs, max_new_tokens=200)  # Adjust this value as needed

# Decode the generated tokens to a string
generated_query = tokenizer.decode(generated_tokens[0])

# Output the result
print(generated_query)

<bos>

첫번째 문장은 사용자에 대한 정보입니다. 다음 text을 기반으로 꼬리 질문을 생성해주세요 단 질문은 해당 글에서 정답을 찾을수 있으면 안됩니다:
조건이 있습니다.
text에서 정답을 찾으면 안됩니다.

text:
SM 직무의 인성면접에 참가하게 된 신입 지원자입니다. 음 제가 팀을 이루어서 스터디를 하고 있는 것은 없지만 제 개인적으로는 아이티 분야와 인공 지능 분야에 대해서 어 어떤 공부를 지금 시작해서 해 나가고 있습니다. 제가 바뀌어 나가는 사회 세상에 대해서 관심을 가지게 되다 보니까 아이티 분야 인공 지능 분야가 굉장히 앞으로는 필요한 분야고 제가 따로 공부를 해두지 않으면 따라가기 어려운 분야라는 생각이 들어서 공부를 해야겠다고 결심을 하게 됐고 하나씩 하나씩 공부를 해 나가고 있습니다. 바뀌어나가고 알 수 없는 단어들이나 알 수 없는 변화들이 계속 제 주위에 나타나는데 그것을 외면하면서 살 수는 없더라구요. 그래서 하나씩 하나씩 제가 처음 들어오는 단어에 대해서 관심을 가지고 그것이 무엇인지 찾아가다 보니까 결국은 깊이 있는 공부가 필요하다는 생각이 들었고 지금은 깊이 있게 공부를 하기 위해서 우선 관련 분야 책을 읽는 것으로 공부를 시작하고 있습니다.,저는 아이티 분야와 인공 지능 분야에 대해 따로 공부를 해두지 않으면 따라가기 어려운 분야라는 생각이 들어서 공부를 해야겠다고 결심을 하게 됐습니다. 알 수 없는 변화들이 계속 나타나는데 외면하면서 살 수는 없어서 깊이 있게 공부를 하기 위해서 우선 관련 분야 책을 읽는 것으로 공부를 시작하고 있습니다.

질문:

SM 직무의 신입 지원자가 공부를 시작한 분야는 무엇일까요?<eos>
